# Email-handling

Send automated emails to the participants of the sports week.

In [1]:
import helper_functions as hf
from helper_functions.classes.player import Player
from helper_functions.setup.email_handling import get_email_address
import markdown
import numpy as np
from pathlib import Path
import pandas as pd
import smtplib, ssl
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from bs4 import BeautifulSoup
from email import encoders
from typing import Callable, Literal
import yaml

hf.LOGGER.setLevel("INFO")
hf.LOGGER.info("Hello there! Logging is active.")


[sports_week: WARNING] - Couldn't load tennis_A2_B2.
	A: Stark Otter vs B: Animated Yak
[sports_week: WARNING] - Couldn't load tennis_A1_B1.
	A: Outrageous Gar vs B: Open Iguana
[sports_week: WARNING] - Couldn't load tennis_B6_C6.
	B: Unsteady Aardvark vs C: Writhing Beaver
[sports_week: WARNING] - Couldn't load tennis_B2_C1.
	B: Animated Yak vs C: Eager Affenpinscher
[sports_week: WARNING] - Couldn't load tennis_B1_C4.
	B: Open Iguana vs C: Arctic Milkfish
[sports_week: WARNING] - Couldn't load tennis_A3_B6.
	A: Fearless Wrasse vs B: Unsteady Aardvark
[sports_week: INFO] - Hello there! Logging is active.


In [17]:
FPATH_SENT_MAILS = hf.FpathRegistry.get_path_hidden().parent.joinpath("mails_sent.yml")
if not FPATH_SENT_MAILS.exists() or yaml.safe_load(FPATH_SENT_MAILS.read_text(encoding="utf-8")) is None:
    FPATH_SENT_MAILS.parent.mkdir(parents=True, exist_ok=True)
    FPATH_SENT_MAILS.write_text(yaml.safe_dump({"fee_reminder": [], "schedule": [], "confirmation_reminder": [], "schedule_update": []}), encoding="utf-8")

def get_sent_mails(key: Literal["fee_reminder", "schedule", "confirmation_reminder", "schedule_update"]) -> list[str]:
    """Load the sent mails from the YAML file."""
    return yaml.safe_load(FPATH_SENT_MAILS.read_text(encoding="utf-8"))[key]

def update_sent_mails(key: Literal["fee_reminder", "schedule", "confirmation_reminder", "schedule_update"], new_name: str):
    """Update the sent mails in the YAML file."""
    assert new_name in hf.DATA_NOW.nickname_to_name_df["nickname"].tolist(), f"{new_name} is not a valid nickname."
    sent_mails = yaml.safe_load(FPATH_SENT_MAILS.read_text(encoding="utf-8"))
    if new_name in sent_mails[key]:
        print(f"{new_name} already in {key}.")
        return
    sent_mails[key].append(new_name)
    FPATH_SENT_MAILS.write_text(yaml.safe_dump(sent_mails), encoding="utf-8")

# update_sent_mails("fee_reminder", "Pushy Bulldog")


## Sending out the emails

Following tutorial at https://realpython.com/python-send-email/ to send out mails programmatically.

In [19]:

def write_email_to_player(player_nick: str, 
                          player_email: str, 
                          email_text: str, 
                          email_subject: str,
                          sender_email: str, 
                          password: str, 
                          yaml_key: Literal["fee_reminder", "schedule", "confirmation_reminder", "schedule_update"],
                          im_path: Path | None = None,
                          force=False,
                          write_to_cc: bool = False,
                          ) -> bool:
    if not force and player_nick in get_sent_mails(yaml_key):
        hf.LOGGER.info(f"Mail to {player_nick} already sent. Skipping.")
        return False

    html_text = markdown.markdown(email_text.replace("\\", "<br>"))
    soup = BeautifulSoup(html_text, 'html')
    plain_text = soup.get_text()
    # Turn these into plain/html MIMEText objects
    message = MIMEMultipart("alternative")
    message["Subject"] = email_subject
    message["From"] = sender_email
    message["To"] = player_email

    part1 = MIMEText(plain_text, "plain")
    part2 = MIMEText(html_text, "html")

    if im_path is not None:
        # Open png file in binary mode
        with open(im_path, "rb") as attachment:
            # Add file as application/octet-stream
            # Email client can usually download this automatically as attachment
            part3 = MIMEBase("application", "octet-stream")
            part3.set_payload(attachment.read())

        # Encode file in ASCII characters to send by email    
        encoders.encode_base64(part3)

        # Add header as key/value pair to attachment part
        part3.add_header(
            "Content-Disposition",
            f"attachment; filename= {im_path.name}",
        )
        message.attach(part3)

    # Add HTML/plain-text parts to MIMEMultipart message
    # The email client will try to render the last part first
    message.attach(part1)
    message.attach(part2)

    # - Actually send the mail
    hf.LOGGER.info(f"Sending mail to {player_nick}...")
    port = 587
    # Create secure connection with server and send email
    context = ssl.create_default_context()
    recipients = [player_email]
    if write_to_cc:
        recipients.append(sender_email)
    with smtplib.SMTP("ssmtp.mpe.mpg.de", port) as server:
        server.starttls(context=context) 
        server.login("fbalzer", password)
        server.sendmail(
            sender_email, recipients, message.as_string()
        )
    update_sent_mails(yaml_key, player_nick)
    return True
# for nickname in df.sort_values("response_timestamp")["nickname"]:
#     write_email_to_player(df.loc[nickname], sender_email, password, force=False)



In [4]:
# LOG IN

SENDER_PASSWORD = input("Type your password and press enter:")
SENDER_MAIL = "@".join(["fbalzer", "mpe.mpg.de"])  # slight protection against sp am



### Part 1: Reminder for fee payment

In [5]:
HIDDEN_DF = hf.FpathRegistry.get_hidden_responses().set_index("nickname", drop=False)


In [6]:
unpaid = HIDDEN_DF[~HIDDEN_DF["has_paid_fee"]]
len(HIDDEN_DF), len(unpaid)


(186, 13)

In [7]:
def get_reminder_text(player: pd.Series) -> tuple[str, str]:
    base_text = hf.DATAPATH.joinpath("helper_texts/email_1_reminder_text.md").read_text(encoding="utf-8")
    first_name, nickname = player["name"].split()[0],  player["nickname"]
    text = base_text.format(first_name=first_name, nickname=nickname)
    email = player["email"]
    html_text = markdown.markdown(text.replace("\\", "<br>"))
    return email, html_text


def _send_fee_reminder(player: pd.Series):
    player_email, email_text = get_reminder_text(player)
    subject = "[SPORTS WEEK]: Fee Reminder"
    write_email_to_player(player["nickname"], player_email, email_text, subject, SENDER_MAIL, SENDER_PASSWORD, "fee_reminder", force=False)


In [ ]:
print(SENDER_MAIL.split("@")[1])  # Will raise exception if not logged in
for _, player in unpaid.iterrows():
    _send_fee_reminder(player)


In [ ]:
df = hf.DATA_NOW.players.set_index("nickname", drop=False)

df = df.join(HIDDEN_DF[[col for col in HIDDEN_DF.columns if not col in df.columns]], how="left")
# Filter out any people that are not part of any sports whatsoever (usually shouldn't be the case)
df = df[np.any([df[f"subteam_{s}"] != "" for s in hf.DATA_NOW.sport_events], axis=0)]


192

In [60]:
def _load_links() -> dict[str, str]:
    file = hf.FpathRegistry.get_path_hidden().joinpath("links.yml").read_text(encoding="utf-8")
    return yaml.safe_load(file)

def _get_payment_text(has_paid: bool):
    if has_paid:
        return "💸 We have received your 2 € fee payment, thanks a lot!"
    email = _load_links()["paypal_address"]
    return f"💸 Please pay the 2 € participation fee before the sports week starts, e.g. via PayPal to {email}! Thanks in advance."

def get_initial_schedule_text_for_player(player: pd.Series) -> tuple[str, str, Path]:
    email_base_text = hf.DATAPATH.joinpath("helper_texts/email_2_schedule.md").read_text(encoding="utf-8")
    first_name = player["name"].split()[0]
    nickname = player["nickname"]
    team_name = player["Team"]
    cloth_color = {"Team A": "dark", "Team B": "white", "Team C": "colorful"}[team_name]
    email = get_email_address(player["email"])
    payment_text = _get_payment_text(player["has_paid_fee"])
    links = _load_links()
    
    player_obj = Player.from_series(player, hf.DATA_NOW.matches)
    schedule = player_obj.get_schedule_for_mail().replace(f", {email}", "")
    fpath = hf.DATAPATH.joinpath(f"assets/animal_pics/full_size/{player["nickname"].lower().replace(" ", "_")}.png")
    text = email_base_text.format(first_name=first_name, nickname=nickname, schedule=schedule, team_name=team_name, cloth_color=cloth_color, signal_link=links["signal"], payment_text=payment_text, datashare_link=links["datashare_view"])
    html_text = markdown.markdown(text.replace("\\", "<br>"))
    return email, html_text, fpath


In [ ]:

player = "Dirty Flounder"
# address, text, fpath = get_email_text_for_player(player)
email, text, fpath = get_initial_schedule_text_for_player(df.loc[player])
# print(email, "\n", fpath.name, "\nSports week (Apr 29 - May 3): Your schedule and information")
# hf.copy_to_clipboard(text)
write_email_to_player(player, email, text, "[SPORTS WEEK]: Your Schedule (PLEASE CONFIRM)", SENDER_MAIL, SENDER_PASSWORD, "schedule", fpath, force=False)
# text


[sports_week: INFO] - Sending mail to Dirty Flounder...
Dirty Flounder already in schedule.


True

In [38]:
schedule_sent = get_sent_mails("schedule")
subdf = df[~df["nickname"].apply(lambda x: x in schedule_sent)]
for player, player_row in subdf.iterrows():
    email, text, fpath = get_initial_schedule_text_for_player(player_row)
    # print(email, "\n", fpath.name, "\nSports week (Apr 29 - May 3): Your schedule and information")
    # hf.copy_to_clipboard(text)
    write_email_to_player(player, email, text, "[SPORTS WEEK]: Your Schedule (PLEASE CONFIRM)", SENDER_MAIL, SENDER_PASSWORD, "schedule", fpath, force=False)


[sports_week: INFO] - Sending mail to Our Pheasant...


In [83]:


def get_schedule_update_text_for_player(player: pd.Series) -> tuple[str, str]:
    email_base_text = hf.DATAPATH.joinpath("helper_texts/email_2_schedule_update.md").read_text(encoding="utf-8")
    first_name = player["name"].split()[0]
    nickname = player["nickname"]
    email = get_email_address(player["email"])
    
    player_obj = Player.from_series(player, hf.DATA_NOW.matches)
    schedule = player_obj.get_schedule_for_mail().replace(f", {email}", "")
    text = email_base_text.format(first_name=first_name, nickname=nickname, schedule=schedule)
    html_text = markdown.markdown(text.replace("\\", "<br>"))
    return email, html_text

hf.DATA_NOW.reload()
df = hf.DATA_NOW.players.set_index("nickname", drop=False)
df = df.join(HIDDEN_DF[[col for col in HIDDEN_DF.columns if not col in df.columns]], how="left")
# # Filter out any people that are not part of any sports whatsoever (usually shouldn't be the case)
df = df[np.any([df[f"subteam_{s}"] != "" for s in hf.DATA_NOW.sport_events], axis=0)]

# This one should be done on a case-by-case basis and be marked in the changelog
player = "Trivial Uguisu"
# address, text, fpath = get_email_text_for_player(player)
email, text = get_schedule_update_text_for_player(df.loc[player])
# print(text)
# if write_email_to_player(player, email, text, "[SPORTS WEEK]: Schedule Update", SENDER_MAIL, SENDER_PASSWORD, "schedule_update", force=False):
#     hf.write_changelog_entry(f"Sent schedule update to {player}.")


In [90]:

def get_reminder_email_text(player: pd.Series) -> tuple[str, str]:
    email_base_text = hf.DATAPATH.joinpath("helper_texts/email_3_follow_up_text.md").read_text(encoding="utf-8")
    first_name = player["name"].split()[0]
    nickname = player["nickname"]
    email = get_email_address(player["email"])
    links = _load_links()
    
    player_obj = Player.from_series(player, hf.DATA_NOW.matches)
    schedule = player_obj.get_schedule_for_mail().replace(f", {email}", "")
    text = email_base_text.format(first_name=first_name, nickname=nickname, schedule=schedule, signal_link=links["signal"])
    html_text = markdown.markdown(text.replace("\\", "<br>"))
    return email, html_text

# print(get_reminder_email_text(df.loc["Pushy Bulldog"])[1])
reminder_sent = get_sent_mails("confirmation_reminder")
not_already_reminded = ~df["nickname"].apply(lambda x: x in reminder_sent)
no_confirmation = ~df["has_confirmed"]
no_dropout = ~df["is_full_dropout"]
not_late = ~df["late_entry"]  # For now let's not remind them
subdf = df[not_already_reminded & no_confirmation & no_dropout & not_late]
print(len(subdf), "players to remind")
for player, player_row in subdf.iterrows():
    email, text = get_reminder_email_text(player_row)
    write_email_to_player(player, email, text, "[SPORTS WEEK]: Please confirm your participation", SENDER_MAIL, SENDER_PASSWORD, "confirmation_reminder", force=False)
    # print(player)


59 players to remind
[sports_week: INFO] - Sending mail to Kooky Snake...
[sports_week: INFO] - Sending mail to Far-flung Sloth...
[sports_week: INFO] - Sending mail to Defiant Tortoise...
[sports_week: INFO] - Sending mail to Virtuous Pika...
[sports_week: INFO] - Sending mail to Scientific Angelfish...
[sports_week: INFO] - Sending mail to Worthy Aurochs...
[sports_week: INFO] - Sending mail to Anguished Dragonfly...
[sports_week: INFO] - Sending mail to Reckless Swan...
[sports_week: INFO] - Sending mail to Courageous Bullfrog...
[sports_week: INFO] - Sending mail to Nervous Badger...
[sports_week: INFO] - Sending mail to Nonstop Cuscus...
[sports_week: INFO] - Sending mail to Gripping Penguin...
[sports_week: INFO] - Sending mail to Upset Cheetah...
[sports_week: INFO] - Sending mail to Realistic Coral...
[sports_week: INFO] - Sending mail to Grimy Newfoundland...
[sports_week: INFO] - Sending mail to Unfortunate Salamander...
[sports_week: INFO] - Sending mail to Sane Wallaby...
[